In [1]:
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px

In [16]:
data = pd.read_csv("data/simulated_data.csv")
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data.tail()

In [ ]:
with st.sidebar:
    st.title('🥳 Happiness Dashboard')
    
    year_list = list(data.Date.dt.year.unique())[::-1]
    
    selected_year = st.selectbox('Select a year', year_list, index=len(year_list)-1)
    df_selected_year = data[data.Date.dt.year == selected_year]
    df_selected_year_sorted = df_selected_year.sort_values(by="Happiness_Level", ascending=False)

    color_theme_list = ['blues', 'cividis', 'greens', 'inferno', 'magma', 'plasma', 'reds', 'rainbow', 'turbo', 'viridis']
    selected_color_theme = st.selectbox('Select a color theme', color_theme_list)

In [ ]:
data

In [ ]:
# Plots
# Time Series der Happiness über die Zeit
import plotly.express as px
fig = px.scatter(data, x='Date', y='Happiness_Level',
                 trendline="rolling", trendline_options=dict(window=100),
                 trendline_color_override='Steelblue',
                 trendline_scope='overall')
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_layout(yaxis_title='Happiness')
fig.update_traces(line=dict(width=6))
fig.update_traces(showlegend=False)
fig.show()


In [ ]:
# Funktion zum Plotten des Codes
def plot_happiness_level(df):
    fig = px.scatter(df, x='Date', y='Happiness_Level',
                 trendline="rolling", trendline_options=dict(window=100),
                 trendline_color_override='Steelblue',
                 trendline_scope='overall')
    fig.data = [t for t in fig.data if t.mode == "lines"]
    fig.update_layout(yaxis_title='Happiness')
    fig.update_traces(line=dict(width=6))
    fig.update_traces(showlegend=False)

    return fig

In [29]:
# Histogram für den jeweilgen Monat
import plotly.express as px

fig = px.histogram(data, x='Happiness_Level', 
                   color_discrete_sequence=['steelblue'])
fig.update_layout(bargap=0.1)
fig.update_layout(xaxis_title='Happiness')
fig.update_layout(yaxis_title='')

fig.show()


In [26]:
# Funktion fürs Histogram
def happiness_histogram(df):
    fig = px.histogram(df, x='Happiness_Level', 
                       color_discrete_sequence=['steelblue'])
    fig.update_layout(bargap=0.1)
    fig.update_layout(xaxis_title='Happiness')
    fig.update_layout(yaxis_title='')
    return fig

## Sentiment Analysis

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
df = pd.read_csv('data/simulated_data.csv')
df.head()

In [ ]:
## Feature Engineering
from transformers import pipeline

# load that bert-i-boy
sentiment_pipe = pipeline('sentiment-analysis')

In [ ]:
texts = df['Feedback'].tolist()

In [ ]:
results = sentiment_pipe(texts)

In [ ]:
results[:5], texts[:5]

In [ ]:
# Categorien
import pandas as pd 

df = pd.DataFrame(results)
df['texts'] = texts

def categorize_sentiment(label):
    if label == 'POSITIVE':
        return 'positive'
    elif label == 'NEGATIVE':
        return 'negative'
    else:
        return 'neutral'

In [ ]:
# Apply categorization
df['sentiment'] = df['label'].apply(categorize_sentiment)

# Aggregate results
sentiment_counts = df['sentiment'].value_counts()

In [ ]:
## Word Cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate word clouds for each sentiment
for sentiment in ['positive', 'neutral', 'negative']:
    if 'text' in df.columns:
        text = " ".join(review for review in df[df['sentiment'] == sentiment]['text'])
        wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)

        plt.figure()
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.title(f"Word Cloud for {sentiment.capitalize()} Sentiment")
        plt.axis("off")
        plt.show()


In [ ]:
# Create a donut chart
labels = sentiment_counts.index
sizes = sentiment_counts.values
colors = ['#ff9999','#66b3ff','#99ff99']
 
fig1, ax1 = plt.subplots()
ax1.pie(sizes, colors=colors, labels=labels, autopct='%1.1f%%', startangle=90)
 
# Draw a circle at the center to make it look like a donut
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
 
# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.title("Sentiment Distribution")
plt.savefig('sample_plot.png')

plt.show()
Image(filename='sample_plot.png')
